In [1]:
#Power BI Export

import importlib.util
spec = importlib.util.spec_from_file_location("MongoTwitter.models", r"C:\Users\victor.santos2\OneDrive - Grupo Marista\TCC\Twitter Sentiment Analysis\MongoTwitter\models.py")
models = importlib.util.module_from_spec(spec)
spec.loader.exec_module(models)
import pandas as pd
import mongoengine

mongoengine.register_connection(alias='core',name='sentimentAnalysis')
#1


In [2]:
candidates = [('lhmandetta',2726301380,'Mandetta'),
('EduardoLeite_',123891446,'Eduardo Leite'),
('DatenaOficial',68722955, 'Datena'),
('cirogomes',33374761, 'Ciro Gomes'),
('jdoriajr',64706049, 'Dória'),
('LulaOficial',2670726740,'Lula'),
('jairbolsonaro',128372940, 'Bolsonaro'),]
can_df = pd.DataFrame(data=candidates, columns=['Handle', 'author_id','Name'])
can_df
#2


,Handle,author_id,Name
0,lhmandetta,2726301380,Mandetta
1,EduardoLeite_,123891446,Eduardo Leite
2,DatenaOficial,68722955,Datena
3,cirogomes,33374761,Ciro Gomes
4,jdoriajr,64706049,Dória
5,LulaOficial,2670726740,Lula
6,jairbolsonaro,128372940,Bolsonaro


In [3]:
objs_tweets = models.Tweets.objects().all().values_list('id', 'tweet_id','author_id','created_at','text','lang','retweet_count','reply_count','like_count','quote_count')
tweets = pd.DataFrame.from_records(data=objs_tweets, columns=['id','tweet_id','author_id','created_at','text','lang','retweet_count','reply_count','like_count','quote_count'])
tweets_df = tweets
tweets = tweets.drop(columns=['id','created_at','text','lang','retweet_count','reply_count','like_count','quote_count'])
tweets = tweets.rename(columns={"tweet_id": "conversation_id"})
tweets = pd.merge(tweets,can_df, how='left')
# tweets = tweets.merge(can_df)
tweets = tweets.drop(columns=['author_id'])
tweets
# ratings.loc[ratings.movie_id == 1, :].head()
# tweets.loc[tweets.Handle == 'NaN', :].head()
#3

,conversation_id,Handle,Name
0,1437158653623160837,LulaOficial,Lula
1,1437054980884684804,LulaOficial,Lula
2,1436397090003881984,LulaOficial,Lula
3,1436156966817181698,LulaOficial,Lula
4,1435999486162186244,LulaOficial,Lula
...,...,...,...
621,1440418824789000203,jairbolsonaro,Bolsonaro
622,1440355742372937731,jairbolsonaro,Bolsonaro
623,1440294198595391488,jairbolsonaro,Bolsonaro
624,1440272143254581249,jairbolsonaro,Bolsonaro


In [4]:
tweets_sentiments = models.TweetsSentiment.objects.all().values_list('tweet_id','length','sentiment', 'negative','positive', 'neutral','clean_text')
tweets_sentiments_df = pd.DataFrame.from_records(data=tweets_sentiments, columns=['tweet_id','length','sentiment', 'negative','positive', 'neutral','clean_text'])
tweets_df = pd.merge(tweets_df,tweets_sentiments_df)
#4
tweets_df


,id,tweet_id,author_id,created_at,text,lang,retweet_count,reply_count,like_count,quote_count,length,sentiment,negative,positive,neutral,clean_text
0,613ec5589eb2bd97646d86ad,1437158653623160837,2670726740,2021-09-12 20:58:18,@dilmabr 🙏🏼,und,123,76,6411,7,3,neutral,0.05,0.09,0.86,🙏🏼
1,613ec5599eb2bd97646d86ae,1437054980884684804,2670726740,2021-09-12 14:06:21,Recomendação de leitura para o domingo: Amor à...,pt,1812,1529,18236,271,104,positive,0.00,1.00,0.00,recomendação de leitura para o domingo: amor à...
2,613ec5599eb2bd97646d86af,1436397090003881984,2670726740,2021-09-10 18:32:07,"“Olho pra trás, vejo a estrada que eu trilhei,...",pt,783,538,7294,84,125,negative,0.62,0.02,0.36,"“olho pra trás, vejo a estrada que eu trilhei,..."
3,613ec5599eb2bd97646d86b0,1436156966817181698,2670726740,2021-09-10 02:37:58,"@leandraleal Feliz Aniversário, companheira! 💫",pt,87,84,7371,7,32,positive,0.00,0.99,0.01,"feliz aniversário, companheira!"
4,613ec55a9eb2bd97646d86b1,1435999486162186244,2670726740,2021-09-09 16:12:11,@MussumAlive @manobrown 👊🏼,und,10,15,636,1,4,neutral,0.05,0.09,0.86,👊🏼
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614,61524191e51e9c281f6adc2c,1440679821009317892,128372940,2021-09-22 14:10:10,- Mais informações todos os dias em nosso Tele...,pt,6408,11503,31714,14096,51,neutral,0.01,0.04,0.95,- mais informações todos os dias em nosso tele...
615,61524191e51e9c281f6adc2d,1440616737997660160,128372940,2021-09-22 09:59:30,"- O projeto, oferecerá capacitação a empreende...",pt,1249,218,7125,16,131,positive,0.01,0.91,0.08,"- o projeto, oferecerá capacitação a empreende..."
616,61524192e51e9c281f6adc2e,1440616456350158852,128372940,2021-09-22 09:58:23,"- O Ministério da Ciência, Tecnologia e Inovaç...",pt,3068,792,17246,58,215,positive,0.01,0.60,0.39,"- o ministério da ciência, tecnologia e inovaç..."
617,61524193e51e9c281f6adc2f,1440484432352190464,128372940,2021-09-22 01:13:46,- Obrigado brasileiros de Nova Iorque.\n- Rumo...,pt,8735,2894,44811,672,38,positive,0.01,0.94,0.05,- obrigado brasileiros de nova iorque.


In [5]:
objs_replies = models.Replies.objects.all().values_list('id','reply_id', 'in_reply_to_user_id', 'conversation_id' ,'author_id','created_at','text','lang','retweet_count','reply_count','like_count','quote_count')
replies_df = pd.DataFrame.from_records(data=objs_replies, columns=['id','reply_id', 'in_reply_to_user_id', 'conversation_id' ,'author_id','created_at','text','lang','retweet_count','reply_count','like_count','quote_count'])
#5
replies_df


,id,reply_id,in_reply_to_user_id,conversation_id,author_id,created_at,text,lang,retweet_count,reply_count,like_count,quote_count
0,613ec38d31bdd2aa9c1ac570,1437254908391174151,620196046,1437122304241291264,1148765022899900417,2021-09-13 03:20:47,@paulo_a_andrade @jairbolsonaro 😂 😂 😂 😂 😂 😂 😂 ...,und,0,0,0,0
1,613ec38d31bdd2aa9c1ac572,1437254749422854152,852146729256460288,1437122304241291264,1436748217060892675,2021-09-13 03:20:09,@fabiohpires @KatiaAMBarros @arnonco @jairbols...,pt,0,0,0,0
2,613ec38d31bdd2aa9c1ac574,1437254632791699460,1260784158546309121,1437122304241291264,137197458,2021-09-13 03:19:42,@Antonio40867082 @jairbolsonaro @allanldsantos...,und,0,0,0,0
3,613ec38d31bdd2aa9c1ac578,1437254201105657862,1431434805762301958,1437122304241291264,1045530863277146112,2021-09-13 03:17:59,@luke_pallas @JosCamp61559152 @ClaudioMaduro @...,pt,0,0,0,0
4,613ec38d31bdd2aa9c1ac579,1437254182638129153,1057598413259321344,1437122304241291264,1148765022899900417,2021-09-13 03:17:54,@Adriana43066198 @Carolbolsomina @jairbolsonar...,und,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
103873,61524321e51e9c281f6b0db8,1440646790940815361,128372940,1440259056925511684,1277418261269950464,2021-09-22 11:58:55,@jairbolsonaro Esse povinho baixa da esquerda ...,pt,0,0,0,0
103874,61524321e51e9c281f6b0db9,1440646592076259335,1316896387397812224,1440259056925511684,1167437341,2021-09-22 11:58:08,@BarbaraNicia @MgkMariana @jairbolsonaro Volta...,pt,0,1,1,0
103875,61524321e51e9c281f6b0dba,1440646476200235010,3347747457,1440259056925511684,3433074676,2021-09-22 11:57:40,@MgkMariana @jairbolsonaro Sim o americano cid...,pt,0,1,2,0
103876,61524321e51e9c281f6b0dbb,1440646433749622787,128372940,1440259056925511684,165997795,2021-09-22 11:57:30,@jairbolsonaro Brasilll mitooooo,pt,0,0,0,0


In [6]:
replies_sentiments = models.RepliesSentiment.objects.all().values_list('reply_id','length','sentiment', 'negative','positive', 'neutral','clean_text')
replies_sentiments_df = pd.DataFrame.from_records(data=replies_sentiments, columns=['reply_id','length','sentiment', 'negative','positive', 'neutral','clean_text'])
replies_df = replies_df.merge(replies_sentiments_df)
replies_df
#6

,id,reply_id,in_reply_to_user_id,conversation_id,author_id,created_at,text,lang,retweet_count,reply_count,like_count,quote_count,length,sentiment,negative,positive,neutral,clean_text
0,613ec38d31bdd2aa9c1ac570,1437254908391174151,620196046,1437122304241291264,1148765022899900417,2021-09-13 03:20:47,@paulo_a_andrade @jairbolsonaro 😂 😂 😂 😂 😂 😂 😂 ...,und,0,0,0,0,NaN,positive,0.00,1.00,0.00,😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 ...
1,613ec38d31bdd2aa9c1ac572,1437254749422854152,852146729256460288,1437122304241291264,1436748217060892675,2021-09-13 03:20:09,@fabiohpires @KatiaAMBarros @arnonco @jairbols...,pt,0,0,0,0,NaN,positive,0.00,1.00,0.00,relaxa meu bom kkkk eu tenho a solução
2,613ec38d31bdd2aa9c1ac578,1437254201105657862,1431434805762301958,1437122304241291264,1045530863277146112,2021-09-13 03:17:59,@luke_pallas @JosCamp61559152 @ClaudioMaduro @...,pt,0,0,0,0,NaN,negative,0.89,0.11,0.00,"a amigao, quero nem saber, se apertar 13 t..."
3,613ec38d31bdd2aa9c1ac579,1437254182638129153,1057598413259321344,1437122304241291264,1148765022899900417,2021-09-13 03:17:54,@Adriana43066198 @Carolbolsomina @jairbolsonar...,und,0,0,0,0,NaN,positive,0.00,1.00,0.00,😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂 😂...
4,613ec38d31bdd2aa9c1ac57c,1437254015675506693,59322035,1437122304241291264,1279799019229917185,2021-09-13 03:17:14,@andreuchoa1 @LaurinhaIronic @jairbolsonaro Se...,pt,0,0,0,0,NaN,mixed,0.43,0.51,0.06,ser alienado!
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93036,61524321e51e9c281f6b0db3,1440649600444993538,128372940,1440259056925511684,1059457984076660737,2021-09-22 12:10:05,@jairbolsonaro Ainda que fosse uma pessoa já v...,pt,0,0,0,0,49.0,mixed,0.69,0.26,0.05,ainda que fosse uma pessoa já vale o sacrifício.
93037,61524321e51e9c281f6b0db4,1440649532597886977,1438538863492935686,1440259056925511684,724720304,2021-09-22 12:09:49,@Kakazinhokampos @jairbolsonaro Kkkkkkk,und,0,0,0,0,9.0,neutral,0.04,0.09,0.87,kkkkkkk
93038,61524321e51e9c281f6b0db5,1440647942147248128,1316896387397812224,1440259056925511684,724720304,2021-09-22 12:03:30,@BarbaraNicia @MgkMariana @jairbolsonaro Sabe ...,pt,0,1,0,0,16.0,neutral,0.12,0.08,0.80,sabe ler não?
93039,61524321e51e9c281f6b0db6,1440647592346472453,128372940,1440259056925511684,1414623178421424139,2021-09-22 12:02:06,@jairbolsonaro Esses acéfalos só pensando no s...,pt,0,0,0,0,50.0,mixed,0.36,0.53,0.11,esses acéfalos só pensando no seu próprio umb...


In [18]:
replies_df = replies_df.merge(tweets)
tweets_df = tweets_df.merge(can_df)
exclude_tweets = [1440468001661198346,1440355742372937731,1440272143254581249,1440259056925511684,1435367536015593486,1435349563062243331]
for i in exclude_tweets:
    replies_df = replies_df.drop(replies_df[replies_df.conversation_id == i].index)
#7
# tweets_df.to_csv('tweets_df.csv')
old_text = tweets_df.clean_text
new_text = ''
for text in old_text:
    tokens = text.split()
    for token in tokens:
        if token not in ['o','a','os','as','de','da','do','dos','das','em','na','no']:
            new_text += f' {token} '

print(new_text)

 🙏🏼  recomendação  leitura  para  domingo:  amor  à  maneira  deus,  meu  amigo  padre  júlio  lancellotti  ().  “olho  pra  trás,  vejo  estrada  que  eu  trilhei,  mó  cota.quem  teve  lado  lado  e  quem  só  ficou  bota…”tamo  1°  com  ,  via  feliz  aniversário,  companheira!  👊🏼  já  ouviram?  bom  dia.  boa  noite!  hoje,  meia  noite.  uma  camisa  bonita  dessas  👊🏼  👀  verdade  sempre  vence.  lembrança  um  outro  brasil  neste  dia  mundial  alfabetização.  brasil  tem  jeito!  link  para  assistir:  lula  fala  ao  brasil  -  assista  hoje  partir  19h  discurso  ex-presidente  lula  ao  país-  acompanhe  transmissão  vivo  nossas  redes-  ajude  divulgar  #7desetembro  ência  verdade  vai  sendo  restabelecida  pouco  pouco.  aniversário  dela  hoje!  amanhã,  8h30.  por  isso,  queremos  ouvir  e  trocar  experiências  com  quem  atua,  pesquisa  ou  trabalha  contra  redes  desinformação  brasil.  é  fundamental  compreender  e  expor  mecanismos  produção  e  difusão  

In [8]:
import pyreadstat
pyreadstat.write_sav(replies_df, 'replies_df_sav')
# pyreadstat.write_sav(replies_df, 'tweets_df_sav')

In [9]:
replies_df.to_csv('replies_df.csv')
tweets_df.to_csv('tweets_df.csv')

In [10]:
obj_summa = models.Summarizations.objects.all().values_list('summarization_id','text', 'rankScore')
summs = pd.DataFrame(data=obj_summa, columns=['summarization_id','text', 'rankScore'])
summs_df = summs.rename(columns={"summarization_id": "tweet_id",'text':'summ_text'})
#8

summs_df

,tweet_id,summ_text,rankScore
0,1437054980884684804,""" vagabundo o próprio lobo, achando que t...",1.00
1,1437054980884684804,"melhora esse marketing, pq esse de bom moço n...",0.89
2,1437054980884684804,"luladrao pensa que engana a quem , esse infe...",0.77
3,1437054980884684804,ao menos dessa vez segurou o livro corretamente.,0.65
4,1437054980884684804,já aprendeu a ler???,0.62
...,...,...,...
7804,1440259056925511684,e suplico ao meu presidente bolsonaro faça oqu...,0.20
7805,1440259056925511684,"a q ponto q nos chegamos, sem escolha p 2022....",0.20
7806,1440259056925511684,vc é piada no mundo seu genocida de cada 10 ...,0.19
7807,1440259056925511684,meia dúzia de acéfalos protestam contra jair b...,0.15


In [11]:
obj = models.Replies.objects.filter(reply_id=1440716418538897419).first()
print('ID tweet:',obj.conversation_id)
twee = models.Tweets.objects.filter(tweet_id=obj.conversation_id).first()


print('Texto tweet',twee.text)

ID tweet: 1440259056925511684
Texto tweet - Meia dúzia de acéfalos protestam contra Jair Bolsonaro para delírio de parte da imprensa ($)  brasileira. https://t.co/jB3lVjVpZG


In [12]:
exclude_tweets = [1440468001661198346,1440355742372937731,1440272143254581249,1440259056925511684,1435367536015593486,1435349563062243331]
for i in exclude_tweets:
    replies_df = replies_df.drop(replies_df[replies_df.conversation_id == i].index)
#9

print(len(replies_df))


90616


In [13]:
replies = models.Replies.objects.all()
len(replies)

103878

In [14]:
i = 0
n = 0
for tweet in tweets_df:
    for reply in replies_df:
        if tweet['tweet_id'] == replies_df['conversation_id']:
            n +=reply.negative
            i += 1
    avg = n/i    
    tweet.RepliesNegative = avg 
tweets_df
    



TypeError: string indices must be integers

In [ ]:
t_list = tweets_df['tweet_id']
n_list = []
for i in replies_df['conversation_id']:
    if i not in t_list:
        if i not in n_list:
            n_list.append(i) 
    else:
        pass
        # print('no')

print(len(n_list))

444


In [ ]:
exc_reply_df = pd.read_csv("../../SPSS/conv.csv")
r_list = []

for i in exc_reply_df['conversation_id']:
    if i in tweets_df['tweet_id']:
        if i not in exc_reply_df:
            r_list.append(i)

print(len(r_list))

0


In [ ]:
exc_reply_df = pd.read_csv("../../SPSS/data.csv")
exc_reply_df

r_list = []
for i in replies_df.id:
    if i in exc_reply_df.Replies:
        r_list.append(i)

print(len(replies_df))



AttributeError: 'DataFrame' object has no attribute 'Replies'

In [ ]:
reply = models.Replies.objects.filter(reply_id=1435374704710365188).first()

print(reply.conversation_id)


exclude_tweets = [1440468001661198346,1440355742372937731,1440272143254581249,1440259056925511684,1435367536015593486,1435349563062243331]

1435367536015593486


In [ ]:
tweet = models.Tweets.objects.get(tweet_id=1440259056925511684)
print(tweet.text)

- Meia dúzia de acéfalos protestam contra Jair Bolsonaro para delírio de parte da imprensa ($)  brasileira. https://t.co/jB3lVjVpZG


In [ ]:
#Tests
candidates = [('lhmandetta',2726301380),
('EduardoLeite_',123891446),
('DatenaOficial',68722955),
('cirogomes',33374761),
('jdoriajr',64706049),
('LulaOficial',2670726740),
('jairbolsonaro',128372940)]
for candidate in candidates:
    print(candidate[0],'replies:',len(replies_df[replies_df['Handle']==candidate[0]]))
 

lhmandetta replies: 1547
EduardoLeite_ replies: 6436
DatenaOficial replies: 2074
cirogomes replies: 28488
jdoriajr replies: 16020
LulaOficial replies: 18991
jairbolsonaro replies: 19391
